In [ ]:
# m3u8 playlist approach
# Place music fies in "./Music/"
# Using VLC to create a playlist of songs in "./Music"
# Save Playlist as "./Music/Playlist.m3u8"

In [ ]:
# install additional python modules:
#!pip install pygame
#!pip install pydub
#!pip install apscheduler

In [1]:
# Future Ideas

# Choose start time from calling script: python JAudioSync.py "18:55:00"
# Stopping music playback inbetween
# Resume playback from any number of track in "playlist"

# Common interace: distribute commands to all clients at the same time
   # Alternatively copy prewritten commands (for each client IP) to Termux (needs ssh key auth), e.g ssh 192.10.10.2 python JAudioSync.py "18:55:00"



In [ ]:
import pygame.mixer
import os
from urllib.parse import unquote
from datetime import timedelta, datetime
from pydub import AudioSegment
from math import ceil
from apscheduler.schedulers.blocking import BlockingScheduler

# Read .m3u8 playlist file and extract file path to "playlist"
def load_playlist(playlist_file):
    with open(playlist_file, 'r') as file:
        lines = file.readlines()
    # Filter out comments and empty lines, clean, add ./Music
    playlist = [os.path.join("./Music", unquote(line.strip())) for line in lines if line.strip() and not line.startswith('#')]
    return playlist

# Convert time string to a datetime object with date of today
def string_to_datetime(time_string):
    format_str = "%H:%M:%S"
    today_date = datetime.now().date()
    datetime_str = f"{today_date} {time_string}"
    return datetime.strptime(datetime_str, f"%Y-%m-%d {format_str}")

# Get the rounded up playback length of a music file as timedelta (seconds)
def get_music_length(file_path):
    audio = AudioSegment.from_file(file_path)
    length_in_seconds = len(audio) / 1000  # Convert milliseconds to seconds
    rounded_length = ceil(length_in_seconds)
    return timedelta(seconds=rounded_length)

# Load a music file into RAM memory with pygame.mixer.Sound, available globally as "music", enabeling fast playback time compared to streaming from storage
def load_music(music_file):
    global music
    music = pygame.mixer.Sound(music_file)
    print(music_file)
    print("loaded: ", datetime.now().time())
    print(get_music_length(music_file))

# Start playback of music from RAM memory
def play_music():
    music.play()
    print("playing: ", datetime.now().time())

if __name__ == "__main__":
    # Location of playback file, can be changed to root folder
    playlist_file = "./Music/Playlist.m3u8"
    playlist = load_playlist(playlist_file)
    
    # Initializing audio output of pygame.mixer, detects mode automatically, using standard audio interface
    pygame.mixer.init()
    
    # Create a scheduler
    scheduler = BlockingScheduler()

    start_time_str = "18:54:59"
    load_time = string_to_datetime(start_time_str)
    play_time = load_time + timedelta(seconds=1)
    
    # Schedule tasks
    for music_file_path in playlist:
        # Schedule the task at the specified datetime
        scheduler.add_job(load_music, 'date', run_date=load_time, args=[music_file_path])
        scheduler.add_job(play_music, 'date', run_date=play_time)
        music_length = get_music_length(music_file_path)
        load_time = play_time + music_length
        play_time = load_time + timedelta(seconds=1)
    
    # Start the scheduler
    scheduler.start()
